<br>

# Introdução

- https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias


In [ ]:
#!pip3 install requests-ip-rotator

In [ ]:
#!pip3 install traquitanas

In [ ]:
import concurrent
import os
import re
from typing import Literal

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from open_geodata import geo
from requests_ip_rotator import ApiGateway
from selenium.webdriver.common.by import By

from sp_tjsp_divadmin.my_driver import Driver
from sp_tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
)
from sp_tjsp_divadmin.my_paths import (
    adds_path,
    driver_path,
    logs_path,
    output_path_tab,
)

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from traquitanas.scrapping import adds, gecko

In [ ]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

<br>

## Imóveis


Inicialmente montamos uma lista de termos que iremos utiliza para a pesquisa. Juntamos tanto a lista de Unidades, obtida em consulta anterior, como a lista com o nome de municípios.

In [ ]:
# Preciso listar Imóveis para obter o código
df_unidades = pd.read_csv(output_path_tab / 'Unidades.csv')
list_unidades = list(df_unidades['unidades'])

# Passa para lowercase
list_unidades = [x.lower() for x in list_unidades]

# Pega cada palavra
list_unidades_words = [
    word for phrase in list_unidades for word in phrase.split()
]

# Retira duplicados
list_unidades_words = list(set(list_unidades_words))

# Pega maior que 4 caracters
list_unidades_words = [x for x in list_unidades_words if len(x) >= 4]

# Results
print(len(list_unidades_words))
list_unidades_words[:10]

In [ ]:
# Cria Lista
df_geo_mun = geo.load_dataset('tab.sp.tab_municipio_nome')
lista_municipios = list(df_geo_mun['municipio_nome'])

# Passa para lowercase
lista_municipios = [x.lower() for x in lista_municipios]

# Pega cada palavra
lista_municipios_words = [
    word for phrase in lista_municipios for word in phrase.split()
]

# Retira duplicados
lista_municipios_words = list(set(lista_municipios_words))

# Pega maior que 4 caracters
lista_municipios_words = [x for x in lista_municipios_words if len(x) >= 4]

# Results
print(len(lista_municipios_words))
lista_municipios_words[:10]

In [ ]:
list_search = list_unidades_words + lista_municipios_words
list_search = list(set(list_search))

print(len(list_search))
list_search[0:10]

In [ ]:
# Cria Gateway
gateway = ApiGateway(
    site='https://www.tjsp.jus.br',
    access_key_id=aws_access_key_id,
    access_key_secret=aws_secret_access_key,
    regions=['sa-east-1'],
    verbose=True,
)
gateway.pool_connections = 10
gateway.pool_maxsize = 10
gateway.start()

# Cria Session
session = requests.Session()
session.mount(prefix='https://www.tjsp.jus.br', adapter=gateway)

In [ ]:
def get_id_unidade(
    unidade,
    req_type: Literal['basic', 'aws'] = 'aws',
    req_session=session,
) -> pd.DataFrame:
    """
    Obtem o código (ou ID) das Unidades (ou Imóveis) do TJSP

    :param unidade: nome da Unidade, ou trecho do nome
    :return: tabela com os valores
    """
    # Get Data
    if req_type == 'basic':
        r = requests.post(
            'https://www.tjsp.jus.br/AutoComplete/ListarImoveis',
            json={'texto': unidade},
        )
    elif req_type == 'aws':
        r = req_session.post(
            'https://www.tjsp.jus.br/AutoComplete/ListarImoveis',
            json={'texto': unidade},
        )

    # Se é lista vazia, ignora
    if r.json() == 'listaVazia':
        print(f'----------- Vazio em {unidade}')

    # Se não é, registra!
    else:
        try:
            df = pd.DataFrame(r.json())
            df = df.rename(
                mapper={
                    'Codigo': 'id_unidade',
                    'Descricao': 'unidade',
                },
                axis='columns',
            )
            return df
        except:
            print(f'>>>>>>>> Errou em {unidade}')

In [ ]:
get_id_unidade(unidade='Santos', req_type='basic')

Inicialmente eu tentei utilizar um método básico para consultas, utilizando a biblioteca `requests`.

```
list_dfs = []
for i in list_unidades:
    list_dfs.append(get_id_unidade(unidade=i))


df_unidades_ids = pd.concat(objs=list_dfs, ignore_index=True)
df_unidades_ids
```

<br>

Contudo, não estava nada eficente. Rodou trinta minutos e não baixou tudo... dai cancelei e procurei alternativas.

In [ ]:
# Encerra o worker
# gateway.shutdown()

In [ ]:
get_id_unidade(unidade='Santos', req_type='aws')

In [ ]:
# Parameters
MAX_THREADS = 4
list_dfs = []
list_futures = []

# Paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Cria Lista de Tarefas
    for un in list_search:
        futures = executor.submit(get_id_unidade, un)
        list_futures.append(futures)

    # Executa Lista de Tarefas
    for future in concurrent.futures.as_completed(list_futures):
        print(future.result())
        list_dfs.append(future.result())

# Encerra o worker
gateway.shutdown()

In [ ]:
# Junta
df_unidades_ids = pd.concat(objs=list_dfs, ignore_index=True)
df_unidades_ids = df_unidades_ids.drop_duplicates()
df_unidades_ids = df_unidades_ids.sort_values(by='id_unidade', ascending=True)
df_unidades_ids = df_unidades_ids.reset_index(drop=True)

# Resultados
df_unidades_ids.info()
display(df_unidades_ids.head())
display(df_unidades_ids.tail())

Com isso descobrimos que apesar da "força bruta" não foi possível capturar todas as Unidades.


In [ ]:
# Pesquisar uma Unidade Específica
df_unidades_ids[df_unidades_ids['id_unidade'] == 259]

In [ ]:
filename = 'Unidades Id'

# Salva
df_unidades_ids.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_unidades_ids.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)

<br>

---

## Detalhes das Unidades


In [ ]:
def get_detalhes_unidades(
    id_unidade,
    req_type: Literal['basic', 'aws'] = 'aws',
    req_session=session,
):
    """
    _summary_
    # Uma vez com o Código do Imóvel, consigo obter detalhes
    parmsEntrada=827&codigoTipoBusca=2
    Aqui tem tudo que eu preciso!!!!
    :param id_unidade: _description_
    :type id_unidade: _type_
    :param req_type: _description_, defaults to 'session'
    :type req_type: Literal[&#39;basic&#39;, &#39;session&#39;], optional
    :param req_session: _description_, defaults to session
    :type req_session: _type_, optional
    :return: _description_
    :rtype: _type_
    """

    if req_type == 'basic':
        r = requests.post(
            'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
            json={'parmsEntrada': id_unidade, 'codigoTipoBusca': 2},
        )

    elif req_type == 'aws':
        r = req_session.post(
            'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
            json={'parmsEntrada': id_unidade, 'codigoTipoBusca': 2},
        )

    soup = BeautifulSoup(r.text, 'html.parser')
    return {'id_unidade': id_unidade, 'soup': soup}

In [ ]:
def get_detalhes_unidades_soup(res_get_detalhes_unidades):
    """
    _summary_

    :param soup: _description_
    :return: _description_
    """
    # Trbalha com input
    soup = res_get_detalhes_unidades['soup']
    id_unidade = res_get_detalhes_unidades['id_unidade']

    # Parâmetros
    try:
        unidade = soup.find(name='h3', attrs={'id': 'imovelNome'}).text.strip()
    except:
        unidade = 'Sem nome'

    endereco = (
        soup.find(string=re.compile('.*Endereço.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    telefone = (
        soup.find(string=re.compile('.*Telefone.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    fax = (
        soup.find(string=re.compile('.*Fax.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    email = (
        soup.find(string=re.compile('.*E-mail.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    cj: str = (
        soup.find(
            string=re.compile('.*Circunscrição Judiciária.*', flags=re.DOTALL)
        )
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    num_varas_instaladas = (
        soup.find(
            string=re.compile('.*Número de Varas Instaladas.*', flags=re.DOTALL)
        )
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    entrancia = (
        soup.find(string=re.compile('.*Entrância.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    # re.compile('.*Comarca.*', flags=re.DOTALL)
    comarca = (
        soup.find('dt', string=re.compile('.*Comarca.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    dist_capital = (
        soup.find(
            string=re.compile('.*Distância da Capital.*', flags=re.DOTALL)
        )
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    tensao_eletrica = (
        soup.find(string=re.compile('.*Tensão Elétrica.*', flags=re.DOTALL))
        .parent.parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    tj_dict = {
        'id_unidade': id_unidade,
        'unidade': unidade,
        'endereco': endereco,
        'telefone': telefone,
        'fax': fax,
        'email': email,
        'cj': cj,
        'num_varas_instaladas': num_varas_instaladas,
        'entrancia': entrancia,
        'comarca': comarca,
        'dist_capital': dist_capital,
        'tensao_eletrica': tensao_eletrica,
    }

    return tj_dict

In [ ]:
id_unidade = 88
results = get_detalhes_unidades(id_unidade=id_unidade, req_type='basic')
soup = results['soup']
#print(soup.prettify())
get_detalhes_unidades_soup(res_get_detalhes_unidades=results)

In [ ]:
# # re.compile('.*Comarca.*', flags=re.DOTALL)
# # re.compile('.*Comarca.*', flags=re.DOTALL)
# comarca = (    
#     soup.find('dt', string=re.compile('.*Comarca.*', flags=re.DOTALL))
#     .parent.parent.find(name='dd')
#     .find(name='span')
#     .text.strip()
# )
# comarca

In [ ]:
#print(soup.prettify())

In [ ]:
def get_unidades(id_unidade=id_unidade):
    soup = get_detalhes_unidades(
        id_unidade=id_unidade, req_type='aws', req_session=session
    )
    dd = get_detalhes_unidades_soup(res_get_detalhes_unidades=soup)
    return dd

In [ ]:
id_unidade = 1
get_unidades(id_unidade=id_unidade)

In [ ]:
# Parameters
MAX_THREADS = 4
list_dfs = []
list_futures = []

# Paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Cria Lista de Tarefas
    for i in range(1, 1500):
        futures = executor.submit(get_unidades, i)
        list_futures.append(futures)

    # Executa Lista de Tarefas
    for future in concurrent.futures.as_completed(list_futures):
        print(future.result())
        list_dfs.append(future.result())

# Encerra o worker
gateway.shutdown()

In [ ]:
pd.DataFrame([{'list': '1', 'fff': '2'}, {'list': '1', 'fff': '2'}])

In [ ]:
df_unidades_infos = pd.DataFrame(list_dfs)
df_unidades_infos = df_unidades_infos.drop_duplicates()
df_unidades_infos = df_unidades_infos.sort_values(by='id_unidade', ascending=True)
df_unidades_infos = df_unidades_infos.reset_index(drop=True)

# Results
df_unidades_infos.info()
display(df_unidades_infos.head())
display(df_unidades_infos.tail())

In [ ]:
# Remove ; do fim da coluna
df_unidades_infos['telefone'] = df_unidades_infos['telefone'].str.replace(r'\;$', '', regex=True)
df_unidades_infos['fax'] = df_unidades_infos['fax'].str.replace(r'\;$', '', regex=True)

In [ ]:
# Aplica strip em todo o dataframe
df_unidades_infos = df_unidades_infos.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
mask = (
    (df_unidades_infos['unidade'] == '')
    & (df_unidades_infos['telefone'] == 'Não Informado')
    & (df_unidades_infos['fax'] == 'Não Informado')
    & (df_unidades_infos['email'] == 'Não Informado')
    & (df_unidades_infos['cj'] == 'Não Informado')
    & (df_unidades_infos['entrancia'] == 'Não Informado')
    & (df_unidades_infos['comarca'] == 'Não Informado')
    & (df_unidades_infos['dist_capital'] == 'Não Informado')
    & (df_unidades_infos['tensao_eletrica'] == 'Não Informado')
)
df_unidades_infos = df_unidades_infos[~mask]
df_unidades_infos

In [ ]:
filename = 'Unidades Detalhes e Infos'

# Salva
df_unidades_infos.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_unidades_infos.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)

In [ ]:
# list_tags = soup.find_all(name='dt')
# list_tags
# a = list_tags[2]
# a.find_next_sibling('dd').text.strip()

<br>

## Setores


In [ ]:
def get_id_setor(setor) -> pd.DataFrame:
    """
    Obtem o código (ou ID) dos Setores do TJSP

    :param unidade: _description_
    :return: tabela com os valores
    """
    # Get Data
    r = requests.post(
        'https://www.tjsp.jus.br/AutoComplete/ListarSetores',
        json={'texto': setor},
    )

    # Se é lista vazia, ignora
    if r.json() == 'listaVazia':
        pass

    # Se não é, registra!
    else:
        df = pd.DataFrame(r.json())
        df = df.rename(
            mapper={
                'Codigo': 'id_setor',
                'Descricao': 'setor',
            },
            axis='columns',
        )
        df[['setor', 'unidade']] = df['setor'].str.split('|', expand=True)
        df = df.sort_values(by='id_setor', ascending=True)
        return df


#
df = get_id_setor(setor=list_unidades[0])
df.to_clipboard()
df

In [ ]:
https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca
parmsEntrada=3534&codigoTipoBusca=3

In [ ]:
# Uma vez com o Código do Imóvel, consigo obter detalhes
def get_detalhes_setor(id_setor):
    r = requests.post(
        'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
        json={'parmsEntrada': id_setor, 'codigoTipoBusca': 3},
    )
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup


# parmsEntrada=827&codigoTipoBusca=2
# Aqui tem tudo que eu preciso!!!!

In [ ]:
soup = get_detalhes_setor(id_setor=3534)
print(soup.prettify())

In [ ]:
tel_principal = (
    soup.find(string=re.compile('.*Telefone Principal.*', flags=re.DOTALL))
    .parent.parent.find(name='dd')
    .find(name='span')
    .text.strip()
)

tel_principal

<br>

## Drivers


In [ ]:
driver = Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,
    verify_ssl=False,
)
driver.add_extension_xpath(my_adds_path=adds_path)

In [ ]:
# Go TJSP
URL = 'https://www.tjsp.jus.br/CanaisComunicacao/ListaTelefonica/Index'
driver.get(URL)

<br>

## Juízes Diretores


In [ ]:
def get_rajs_info(raj_xpath):
    """
    Retorna dicionário das Circuncrições Judiciárias

    :param raj_xpath: selenium.element
    :type raj_xpath: _type_
    :return: _description_
    :rtype: dictionary
    """
    # Xpath
    raj_nome_xpath = raj_xpath.find_element(
        By.XPATH, './/p[@style="font-weight: bold"]'
    )
    diretor_xpath = raj_nome_xpath.find_element(
        By.XPATH, "./following-sibling::p"
    )
    email_xpath = diretor_xpath.find_element(By.XPATH, "./following-sibling::p")

    return {
        'raj_nome': raj_nome_xpath.text,
        'juiz_diretor': diretor_xpath.text,
        'email': email_xpath.text,
    }

In [ ]:
# Localiza <div> irmã dos "Juízes Diretores"
juiz_xpath = driver.find_element(By.XPATH, '//*[text()="Juízes Diretores"]//..')
print(juiz_xpath.text)

In [ ]:
# RAJs
rajs_xpath = juiz_xpath.find_element(By.XPATH, './following-sibling::div')

# Comarcas (linhas)
comarcas_xpath = rajs_xpath.find_elements(
    By.XPATH, './/li[@class="list-group-item"]'
)
print(len(comarcas_xpath))

In [ ]:
list_comarcas = []
for comarca_xpath in comarcas_xpath:
    # Comarca
    print(comarca_xpath.text)

    # Procura
    raj_xpath = comarca_xpath.find_element(
        By.XPATH, './../preceding-sibling::div'
    )

    # CJs
    dict_circuncricao = None
    dict_circuncricao = get_rajs_info(raj_xpath)
    dict_circuncricao.update({'comarca_cirscunscricao': comarca_xpath.text})

    # Lista de Comarcas
    list_comarcas.append(dict_circuncricao)

In [ ]:
# Results
list_comarcas[:2]

In [ ]:
# Create Table
df = pd.DataFrame(list_comarcas)

# Results
df.info()
df.head()

<br>

## Adjust Table


In [ ]:
# Adjust Data
df['raj_nome'] = df['raj_nome'].str.replace('–', '-')
df[['raj_sigla', 'raj_regiao']] = df['raj_nome'].str.split(' - ', expand=True)
df[['comarca', 'cj_sigla']] = df['comarca_cirscunscricao'].str.split(
    ' - ', expand=True
)

# Números
df['raj_id'] = df['raj_sigla'].apply(lambda x: keep_numbers(x))
df['cj_id'] = df['cj_sigla'].apply(lambda x: keep_numbers(x))
df['cj_nome'] = df['cj_sigla'].replace(
    'CJ', 'Circunscrição Judiciária', regex=True
)

# Prefixo
df['juiz_diretor'] = (
    df['juiz_diretor']
    .replace('Juiza diretora: ', '', regex=True)
    .replace('Juiz diretor: ', '', regex=True)
)
df['email'] = df['email'].apply(lambda x: find_text_between_parenthesis(x))

# Results
df.info()
df.head()

In [ ]:
list(df.columns)

In [ ]:
df = df[
    [
        # RAJ
        'raj_id',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        'juiz_diretor',
        'email',
        # CJ
        'cj_id',
        'cj_nome',
        'cj_sigla',
        # Comarca
        'comarca',
    ]
]

# Results
df.info()
df.head()

<br>

## Results


<br>

### RAJs


In [ ]:
df_raj = df[
    [
        # RAJ
        'raj_id',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        'juiz_diretor',
        'email',
    ]
]

df_raj = df_raj.drop_duplicates()
df_raj = df_raj.sort_values(by='raj_id')
df_raj = df_raj.reset_index(drop=True)
display(df_raj.head())

df_raj.to_csv(
    output_path_tab / 'Dim_RAJs.csv',
    index=False,
)

df_raj.to_excel(
    output_path_tab / 'Dim_RAJs.xlsx',
    index=False,
    sheet_name='Dim_RAJs',
)

<br>

### CJs


In [ ]:
df_cjs = df[
    [
        # CJ
        'cj_id',
        'cj_nome',
        'cj_sigla',
        'raj_id',
    ]
]

df_cjs.loc[df_cjs['cj_id'] == '', 'cj_id'] = '0'
df_cjs['cj_id'] = df_cjs['cj_id'].copy()
df_cjs['cj_id'] = df_cjs['cj_id'].astype('int')
df_cjs = df_cjs.drop_duplicates()
df_cjs = df_cjs.sort_values(by='cj_id')
df_cjs = df_cjs.reset_index(drop=True)
display(df_cjs.head())

df_cjs.to_csv(
    output_path_tab / 'Dim_CircunscricaoJudiciaria.csv',
    index=False,
)

df_cjs.to_excel(
    output_path_tab / 'Dim_CircunscricaoJudiciaria.xlsx',
    sheet_name='Dim_CircunscricaoJudiciaria',
    index=False,
)

<br>

### Comarcas


In [ ]:
df_comarca = df[
    [
        # Comarca
        'comarca',
        'cj_id',
    ]
]

df_comarca.loc[df_comarca['cj_id'] == '', 'cj_id'] = '0'
df_comarca['cj_id'] = df_comarca['cj_id'].copy()
df_comarca['cj_id'] = df_comarca['cj_id'].astype('int')
df_comarca = df_comarca.drop_duplicates()

df_comarca = df_comarca.rename(
    {
        'comarca': 'comarca_nome_tjsp',
    },
    axis=1,
)

df_comarca = df_comarca.sort_values(by='comarca_nome_tjsp')
df_comarca = df_comarca.iloc[
    df_comarca['comarca_nome_tjsp'].str.normalize('NFKD').argsort()
]
df_comarca = df_comarca.reset_index(drop=True)
display(df_comarca.head())

df_comarca.to_csv(
    output_path_tab / 'Dim_Comarcas_CJs.csv',
    index=False,
)

df_comarca.to_excel(
    output_path_tab / 'Dim_Comarcas_CJs.xlsx',
    index=False,
    sheet_name='Dim_Comarcas_CJs',
)

In [ ]:
comarcas = df['comarca']

print(f'São {len(comarcas)} comarcas')
print(f'São {len(set(comarcas))} comarcas')

In [ ]:
driver.quit()